In [9]:
import os
import pandas as pd

dataset: https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset  \
saved name.basics as names, title.basics as titles, title.ratings as ratings  
* names columns = const, primaryName, birthYear, deathYear, primaryProfession, knownForTitles (array of tconst)   
* titles columns = tconst, titleType, primaryTitle, originalTitle, isAdult(1=adult title), startYear, endYear, runtimeMinutes, genres (string array, max length=3)
* ratings columns =  tconst, averageRating, numVotes

In [10]:
folder = './imdb/'

In [ ]:
# git bash: head -n 1 titles.tsv > title_sample.tsv
# tail -n+2 titles.tsv | shuf -n 1000000 >> title_sample.tsv

In [11]:
files = [os.path.join(folder, file) for file in os.listdir(folder)]
files

['./imdb/names.tsv',
 './imdb/ratings.tsv',
 './imdb/titles.tsv',
 './imdb/title_sample.tsv']

In [33]:
# os.path.basename(files[0]).split('.')[0]

'names_df'

In [34]:
# dfs = {f"{os.path.basename(files[0]).split('.')[0]}_df": pd.read_csv(file, delimiter="\t") for file in files}
# DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.

C:\Users\tammi\AppData\Local\Temp/ipykernel_20664/3900485227.py:2: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  dfs = {f"{os.path.basename(files[0]).split('.')[0]}_df": pd.read_csv(file, delimiter="\t") for file in files}


In [41]:
ratings = pd.read_csv(files[1], delimiter='\t', dtype={'tconst': str, 'averageRating': 'float64', 'numVotes': int}, usecols=['tconst', 'averageRating'])

In [43]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238892 entries, 0 to 1238891
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1238892 non-null  object 
 1   averageRating  1238892 non-null  float64
dtypes: float64(1), object(1)
memory usage: 18.9+ MB


In [14]:
names = pd.read_csv(files[0], delimiter='\t', usecols=['primaryName', 'knownForTitles'])

In [15]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12005705 entries, 0 to 12005704
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   primaryName     object
 1   knownForTitles  object
dtypes: object(2)
memory usage: 183.2+ MB


In [16]:
names.head()

,primaryName,knownForTitles
0,Fred Astaire,"tt0031983,tt0050419,tt0053137,tt0072308"
1,Lauren Bacall,"tt0038355,tt0037382,tt0075213,tt0117057"
2,Brigitte Bardot,"tt0057345,tt0054452,tt0056404,tt0049189"
3,John Belushi,"tt0080455,tt0078723,tt0072562,tt0077975"
4,Ingmar Bergman,"tt0060827,tt0069467,tt0050986,tt0083922"


In [91]:
titles_sample = pd.read_csv(files[3], delimiter='\t')

In [92]:
titles_sample.genres.unique()

array(['\\N', nan], dtype=object)

In [17]:
t_columns = {'tconst': str,
            'titleType': str,
            'primaryTitle': str,
            'originalTitle': str,
            'isAdult': object,
            'startYear': object,
            'endYear': object,
            'runtimeMinutes': object,
            'genres': object
            }

In [80]:
titles = pd.read_csv(files[2], delimiter='\t', dtype=t_columns, usecols=['tconst','isAdult','genres'])

In [81]:
# isAdult == '2020' > https://www.imdb.com/title/tt11868642/
titles.isAdult.unique()

array(['0', '1', '2019', '1981', '2020', '2017', '\\N', '2014', '2005'],
      dtype=object)

In [82]:
titles.drop(titles[titles.isAdult != '0'].index, inplace=True)

In [85]:
# titles[titles.isAdult == '2020']

,tconst,isAdult,genres


In [86]:
titles.drop(['isAdult'], axis=1, inplace=True)

In [87]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9015495 entries, 0 to 9304285
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   tconst  object
 1   genres  object
dtypes: object(2)
memory usage: 206.3+ MB


In [88]:
titles.genres.unique()

array(['\\N'], dtype=object)